In [1]:
# -*- coding: utf-8 -*-

# Form implementation generated from reading ui file 'inter1.ui'
#
# Created by: PyQt5 UI code generator 5.9.2
#
# WARNING! All changes made in this file will be lost!

from PyQt5 import QtCore, QtGui, QtWidgets
import sys
import numpy as np
import cv2


# def draw_rectangle(event, x, y, flags, param):
#     if event == cv2.EVENT_LBUTTONDBLCLK:
#         cv2.circle(frame, (x, y), 11, (0, 0, 255), -1)
    
#   
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
axis = np.float32([[3, 3, -4],[1, 1, 0],[1, 5, 0],[5, 5, 0],[5, 1, 0] ])

objpoints=[]#世界座標3維點
imgpoints=[]#世界座標2維點
objp = np.zeros((11*8,3), np.float32)
objp[:,:2] = np.mgrid[0:11,0:8].T.reshape(-1,2)
images = ["1.bmp", "2.bmp", "3.bmp", "4.bmp", "5.bmp"]

#
def button1(self):
    iml=cv2.imread("imL.png")
    imr=cv2.imread("imR.png")
    iml=cv2.cvtColor(iml,cv2.COLOR_BGR2GRAY)
    imr=cv2.cvtColor(imr,cv2.COLOR_BGR2GRAY)
        
    stereo = cv2.StereoBM_create(numDisparities = 64, blockSize = 9)
        
    disparity=stereo.compute(iml,imr)
    cv2.imshow("disparity",disparity)
    norm_image = cv2.normalize(disparity, None, alpha = 0, beta = 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    cv2.imshow('norm_image', norm_image)
        
def button2(self):
    cap=cv2.VideoCapture('bgsub.mp4')
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
 
        # Display the resulting frame
            cv2.imshow('Frame',frame)
 
            # Press Q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
              break
 
        else:
            print("out")
            break     
    cap.release()
 
    # Closes all the frames
    cv2.destroyAllWindows()

    
    
def button3(self):
    cap=cv2.VideoCapture('featureTracking.mp4')
    
    params = cv2.SimpleBlobDetector_Params()
    
    params.filterByColor=False
    params.blobColor=255
    
    params.filterByArea = True
    
    params.minArea=30.0
    params.maxArea=100.0
    
 
    #Filter by Circularity
    params.filterByCircularity = True
    params.minCircularity = 0.85
 
     # Filter by Convexity
#     params.filterByConvexity = True
#     params.minConvexity = 0.83
 
    
    
    
    detector = cv2.SimpleBlobDetector_create(params)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            fr=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            
            #偵測斑點
            keypoints = detector.detect(fr)
            print(keypoints)
            for i in keypoints:
                pts=i.pt
                x=int(pts[0])
                y=int(pts[1])
                
                cv2.rectangle(frame,(x-6,y-6),(x+6,y+6),(0,0,255),-1)
            
            
            #畫上標記
            #frame = cv2.drawKeypoints(frame, keypoints, frame)
        # Display the resulting frame
            cv2.imshow('Frame',frame)
             
            # Press Q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
 
        else:
            break     
    cap.release()
 
    # Closes all the frames
    cv2.destroyAllWindows()
    
    
    
def button3_2(self):
    track_len = 10
    detect_interval = 5
    tracks = []
    frame_idx = 0

    cap=cv2.VideoCapture('featureTracking.mp4')
    
    params = cv2.SimpleBlobDetector_Params()
    
    params.filterByColor=False
    params.blobColor=255
    
    params.filterByArea = True
    
    params.minArea=30.0
    params.maxArea=100.0
    
 
    #Filter by Circularity
    params.filterByCircularity = True
    params.minCircularity = 0.85
 
     # Filter by Convexity
#     params.filterByConvexity = True
#     params.minConvexity = 0.83
 
    
    
    
    detector = cv2.SimpleBlobDetector_create(params)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            fr=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            if len(self.tracks) > 0:
                img0, img1 = self.prev_gray, fr
                p0 = np.float32([tr[-1] for tr in self.tracks]).reshape(-1, 1, 2)
                p1, st, err = cv2.calcOpticalFlowPyrLK(img0, img1, p0, None, **lk_params)
                
                
            keypoints = detector.detect(fr)
            
            
            
            
    

    
def button4(self):
    for i in images:
        img = cv2.imread(i)
        #轉化灰階
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #find corners
        #image,patternSize,corner[,flag]
        #retval是否檢測出角點
        #corner 角位置點
        ret, corners = cv2.findChessboardCorners(gray, (11, 8), None)
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            #標定
            ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
            #solvePnRansac 魏茲估計演算法
            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            _, rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, mtx, dist)
            imgpt, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
            imgpt = np.int32(imgpt).reshape(-1, 2)
                
            for q in range(4):
                img=cv2.line(img,tuple(imgpt[0]),tuple(imgpt[q+1]),(0,0,255),3)
                img=cv2.line(img,tuple(imgpt[q+1]),tuple(imgpt[(q+1)%4+1]),(0,0,255),3)
                
                
                '''
                for q, w in zip(range(1), range(2,5)):
                    img = cv2.line(img, tuple(imgpt[q]), tuple(imgpt[w]), (0, 0, 255), 3)
                    img = cv2.line(img, tuple(imgpt[q]), tuple(imgpt[(q+1)%4]), (0, 0, 255), 3)
                    #img = cv2.line(img, tuple(imgpt[1+q]), tuple(imgpt[1+(q + 1) % 4]), (0, 0, 255), 3)
                '''
            cv2.namedWindow('Augmented Reality_' + i, cv2.WINDOW_NORMAL)
            cv2.imshow('Augmented Reality_' + i, img)
            cv2.waitKey(500)

        cv2.destroyAllWindows()
                
    
# 
class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(872, 671)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.pushButton = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton.setGeometry(QtCore.QRect(70, 120, 161, 71))
        self.pushButton.setObjectName("pushButton")
        self.pushButton_2 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_2.setGeometry(QtCore.QRect(70, 350, 161, 71))
        self.pushButton_2.setObjectName("pushButton_2")
        self.pushButton_3 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_3.setGeometry(QtCore.QRect(610, 400, 161, 71))
        self.pushButton_3.setObjectName("pushButton_3")
        self.pushButton_4 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_4.setGeometry(QtCore.QRect(610, 120, 161, 71))
        self.pushButton_4.setObjectName("pushButton_4")
        self.pushButton_5 = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton_5.setGeometry(QtCore.QRect(610, 220, 161, 71))
        self.pushButton_5.setObjectName("pushButton_5")
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 872, 25))
        self.menubar.setObjectName("menubar")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.pushButton.setText(_translate("MainWindow", "1.1 disparity"))
        self.pushButton_2.setText(_translate("MainWindow", "2.1 background subtraction"))
        self.pushButton_3.setText(_translate("MainWindow", "4.1 Augmented Reality"))
        self.pushButton_4.setText(_translate("MainWindow", "3.1 preprocessing"))
        self.pushButton_5.setText(_translate("MainWindow", "3.2 video tracking"))
        self.pushButton.clicked.connect(button1)
        self.pushButton_2.clicked.connect(button2)
        self.pushButton_4.clicked.connect(button3)
        self.pushButton_3.clicked.connect(button4)
        
        
app = QtWidgets.QApplication(sys.argv)
MainWindow = QtWidgets.QMainWindow()
ui = Ui_MainWindow()

ui.setupUi(MainWindow) 
MainWindow.show()
sys.exit(app.exec_())

SystemExit: 0

C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
